# Tip Prediction (XGBoost)

**Hardware**: r5.8xlarge (32 CPU, 256 GB RAM)

In [1]:
import numpy as np
import pandas as pd
import xgboost

from sklearn.metrics import mean_squared_error

from ml_utils import MLUtils

This notebook relies on `xgboost` 0.90. As of this writing, that is the newest version of `xgboost` supported by `dask-xgboost`, and this notebook is intended to compliment `tip_dask_xgboost.ipynb`.

In [2]:
print(xgboost.__version__)

0.90


To begin, initialize an `ml_utils` object. This is a small object used to handle naming and storing the model.

In [3]:
ml_utils = MLUtils(
    ml_task='tip',
    tool='scikit',
    model='xgboost',
)

# Load data and feature engineering

In [4]:
%%time
tip_train = ml_utils.read_parquet_dir(f'{ml_utils.taxi_path}/data/ml/tip_train')
tip_train.shape

CPU times: user 2min 5s, sys: 1min 26s, total: 3min 32s
Wall time: 10min 48s


(219889897, 12)

In [5]:
tip_train.head()

,id,pickup_datetime,dropoff_datetime,pickup_taxizone_id,dropoff_taxizone_id,pickup_weekday,pickup_weekofyear,pickup_hour,pickup_minute,pickup_week_hour,passenger_count,tip_fraction
0,a2acf0e861b84476b1ba8b91f08448f4,2016-07-10 06:56:05,2016-07-10 06:59:53,263.0,236.0,6,27,6,56,150,1.0,0.600000
1,bac9b63e96f14ca9a4e9c236d709f5f2,2016-07-10 10:50:18,2016-07-10 10:55:21,142.0,163.0,6,27,10,50,154,5.0,0.226667
2,6ec52187eb4a4ab6ae3ce632030c0322,2016-07-10 10:50:18,2016-07-10 11:08:38,74.0,66.0,6,27,10,50,154,1.0,0.000000
3,21102720ac044762977f91b30f8857a1,2016-07-10 10:50:19,2016-07-10 10:55:38,161.0,142.0,6,27,10,50,154,1.0,0.166667
4,3d002b5aa2404553adc0ad79c776bf32,2016-07-10 10:50:19,2016-07-10 11:08:54,226.0,261.0,6,27,10,50,154,1.0,0.206957


<br>

Training with the full data ran out of memory, so let's take a sample to be able to successfully train

In [6]:
sample = tip_train.sample(frac=0.5, replace=False, random_state=42)
sample.shape

(109944948, 12)

# Train a model

In [7]:
xgb_reg = xgboost.XGBRegressor(
    objective="reg:squarederror",
    learning_rate=0.1,
    max_depth=8,
    n_estimators=100,
    nthread=32
)

In [8]:
%%time
features = ml_utils.tip_vars.features
y_col = ml_utils.tip_vars.y_col

with ml_utils.time_fit():
    model = xgb_reg.fit(X=tip_train[features], y=tip_train[y_col].values)

[16:41:47] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
CPU times: user 1d 2h 45min 20s, sys: 16min 57s, total: 1d 3h 2min 18s
Wall time: 2h 44min 11s


## Save model

Now that we've trained a model, store it in S3 so it can be deployed in the future.

In [9]:
ml_utils.write_model(model)

uploading model to 's3://saturn-titan/nyc-taxi/ml_results/models/tip__scikit__xgboost.pkl'
successfully uploaded model


## Predict on test set

And calculate metrics. Save predictions and metrics to S3. Before doing that, remove the training data from memory.

In [10]:
%%time
tip_test = ml_utils.read_parquet_dir(f'{ml_utils.taxi_path}/data/ml/tip_test')
preds = tip_test[['id', y_col]].copy()
preds.columns = ['id', 'actual']
preds['predicted'] = model.predict(tip_test[features])

CPU times: user 4min 38s, sys: 16.6 s, total: 4min 55s
Wall time: 1min 35s


In [11]:
preds.head()

,id,actual,predicted
0,2e8f402e4dc44f2fae8b9328a237c4d2,0.117647,0.219883
1,5f067a4121244f42bf460867c23b39c9,0.216842,0.219215
2,60e8442d3d434df4959261905a279f55,0.150000,0.212449
3,2d1537ce2ed347778e078eaee7eacd44,0.106250,0.231131
4,13bb8a9ecbd04b559b7b9e40904026b0,0.000000,0.233784


In [12]:
%%time
ml_utils.write_predictions(preds)

Writing predictions to 's3://saturn-titan/nyc-taxi/ml_results/predictions/tip__scikit__xgboost'
Done writing predictions
CPU times: user 9.76 s, sys: 2.92 s, total: 12.7 s
Wall time: 1min 7s


In [13]:
rmse = mean_squared_error(preds.actual, preds.predicted, squared=False)
ml_utils.write_metric_df('rmse', rmse)

,ml_task,tool,model,metric,value,fit_seconds
0,tip,scikit,xgboost,rmse,0.206786,9851.059192
